In [1]:
from pathlib import Path

import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact

In [2]:
DATA_DIR = "../data"
SLR_FILE = "mean_modelled_slr.csv"
DAMAGE_FILE = "mean_damages_rcp85.csv"

data_dir = Path(DATA_DIR)
slr_path = data_dir / SLR_FILE
damage_path = data_dir / DAMAGE_FILE

In [3]:
slr_data = pd.read_csv(slr_path).pivot(index='City', columns='Year', values='SLR')
# display(slr_data)
damage_data = pd.read_csv(damage_path, index_col='City')
damage_data.columns = slr_data.columns
# display(damage_data)
# print(damage_data.index.symmetric_difference(slr_data.index))

In [4]:
@interact(city=slr_data.index)
def plot(city):
    fig, ax = plt.subplots(figsize=(12, 6))

    slr_data.loc[city].plot(label="Sea level rise (left)", ax=ax)
    ax.set_ylabel("Mean modelled SLR (metres)")
    ax.set_title(city)

    ax2 = damage_data.loc[city].plot(secondary_y=True, ax=ax, label="Damages")
    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc=0)
    ax2.set_ylabel("Expected damages under RCP8.5 (million USD (2005))")
    
    f = folium.Figure(width=12*65, height=6*65)
    m = gpd.tools.geocode(city, provider='nominatim', user_agent='test').explore(tiles='CartoDB positron', zoom_start=12)
    m.add_to(f)
    display(f)

interactive(children=(Dropdown(description='city', options=('Abidjan', 'Accra', 'Adelaide', 'Alexandria', 'Alg…